In [ ]:
 и!curl -o ./model.zip https://alphacephei.com/vosk/models/vosk-model-ru-0.22.zip
!unzip model.zip
!mv vosk-model-ru-0.22/ model
!rm -rf model.zip

In [ ]:
!pip3 install SpeechRecognition pydub
!apt-get install ffmpeg
!pip3 install pydub
!pip3 install Vosk

In [ ]:
# importing libraries
import speech_recognition as sr
import os
from os import path
from pydub import AudioSegment
from pydub.silence import split_on_silence

In [ ]:
# files
src = "file.mp3"
dst = "file.wav"

# convert wav to mp3
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

<_io.BufferedRandom name='СЕКРЕТЫ МИЛЛИАРДЕРОВ.wav'>

In [ ]:
# create a speech recognition object
r = sr.Recognizer()


In [ ]:
# a function that splits the audio file into chunks
# and applies speech recognition
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    # open the audio file using pydub
    sound = AudioSegment.from_wav(path)
    # split audio sound where silence is 700 miliseconds or more and get chunks
    chunks = split_on_silence(sound,
        # experiment with this value for your target audio file
        min_silence_len = 500,
        # adjust this per requirement
        silence_thresh = sound.dBFS-14,
        # keep the silence for 1 second, adjustable as well
        keep_silence=500,
    )
    folder_name = "audio-chunks"
    # create a directory to store the audio chunks
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    # process each chunk
    for i, audio_chunk in enumerate(chunks, start=1):
        # export audio chunk and save it in
        # the `folder_name` directory.
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")
        # recognize the chunk
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            # try converting it to text
            try:
                text = r.recognize_google(audio_listened, language='ru-RU')
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
    # return the text for all chunks detected
    return whole_text

In [ ]:
path = "file.wav"
print("\nFull text:", get_large_audio_transcription(path))